In [ ]:
import torch
import torchvision
import os
from torch.utils.data import DataLoader, Dataset, TensorDataset
from PIL import Image, ImageOps
import numpy as np
import shutil
import random
from pathlib import Path


In [ ]:
def list_files_in_folder(image_folder):
    """Lists file names in a given directory"""
    list_of_files = []
    for file in os.listdir(image_folder):
        if os.path.isfile(os.path.join(image_folder, file)):
            list_of_files.append(file)
    return list_of_files

def create_save_dir(direct, name_subdirectory):
    if not os.path.exists(os.path.join(direct, name_subdirectory)):
        print('make dir')
        os.mkdir(os.path.join(direct, name_subdirectory))
    return os.path.join(direct, name_subdirectory)

In [ ]:
# Load and save data to train, validation and test sets
root = './'
dataloader_train = torchvision.datasets.QMNIST(root, train=True, transform=None, target_transform=None, download=True)
dataloader_test = torchvision.datasets.QMNIST(root, train=False, transform=None, target_transform=None, download=True)

train_dir = create_save_dir('./', 'QMNIST_train')
if not os.listdir(train_dir):
    for i, (image,target) in enumerate(dataloader_train): 
        image.save(os.path.join(train_dir,str(target)+'_'+str(i)+'_train'+'.jpg'), "JPEG")
    
test_dir = create_save_dir('./', 'QMNIST_test')
if not os.listdir(test_dir):
    for i, (image,target) in enumerate(dataloader_test): 
        image.save(os.path.join(test_dir,str(target)+'_'+str(i)+'_test'+'.jpg'), "JPEG")
    

In [ ]:
# Take 1/3 from original QMNIST test set for validation set
folder = './QMNIST_test'
valid_dir = create_save_dir('./', 'QMNIST_valid')
if not os.listdir(valid_dir):
    name_list = list_files_in_folder(folder)
    for digit in range(10):
        count = len([elem for elem in name_list if elem[0]==str(digit)])
        to_move = np.around(count/3)
        count_moved=0
        for i in range(len(name_list)):
            if name_list[i][0]==str(digit):
                if count_moved>=to_move:
                    break
                src = os.path.join(folder, name_list[i])
                dst = os.path.join(valid_dir, name_list[i])
                shutil.move(src,dst)
                count_moved+=1
    

In [ ]:
# Choose key instance digit
key_ins_digit = '9'

In [ ]:
# Create separate folder for key instances
def key_instances_to_folder(direct):
    key_ins_dir = create_save_dir(direct, key_ins_digit)
    list_img_names = list_files_in_folder(direct)
    for i in range(len(list_img_names)):
        if list_img_names[i][0]==key_ins_digit:
            src = os.path.join(direct, list_img_names[i])
            dst = os.path.join(key_ins_dir, list_img_names[i])
            shutil.move(src,dst)

key_instances_to_folder(train_dir)
key_instances_to_folder(valid_dir)
key_instances_to_folder(test_dir)

### Create dataset with selected settings

In [ ]:
def auxiliary_function(count_bags, i_range_min, i_range_max, max_num_bags_of_type, dir_qmnist, 
                       max_num_inst_of_type, img_names_list, save_f):
    count_instances = 0
    list_sampled=[]
    for i in range(i_range_min, i_range_max): 
        if count_instances==max_num_inst_of_type: 
            count_bags+=1
            list_sampled=[]
            if count_bags>max_num_bags_of_type: 
                break
            count_instances = 0

        random_name = random.choice(img_names_list)
        while random_name in list_sampled: # not to repeat instances in one bag
            random_name = random.choice(img_names_list)
            if random_name not in list_sampled:
                break
                
        src = os.path.join(dir_qmnist, random_name) 
        bag_folder = create_save_dir(save_f, str(count_bags).zfill(4))

        dst = os.path.join(bag_folder, random_name) 
        shutil.copy(src, dst)     
        list_sampled.append(random_name) 

        count_instances+=1
        last_used_index_in_list = i
    return last_used_index_in_list, count_bags
        

def compose_dataset(dir_qmnist, save_subfolder, num_bags, num_instances_in_bag, percent_key_instances):
    save_f = create_save_dir(save_subfolder, 'positive')
    num_key_ins_per_bag = np.ceil((num_instances_in_bag*percent_key_instances)/100)
    if num_key_ins_per_bag<1:
        print('WARNING!')

    img_names_list = list_files_in_folder(dir_qmnist)
    random.shuffle(img_names_list)

    ''' Positive bags '''
    # Negative instances in positive bags
    count_bags = 0; i_range_min=0; i_range_max=int(1e20)
    max_num_inst_of_type = num_instances_in_bag-num_key_ins_per_bag
    max_num_bags_of_type = np.ceil(num_bags/2)-1
    last_used_index_in_list, _ = auxiliary_function(count_bags, i_range_min, i_range_max, max_num_bags_of_type, 
                                                 dir_qmnist, max_num_inst_of_type, img_names_list, save_f)

    # Key instances in positive bags       
    key_instance_list = list_files_in_folder(os.path.join(dir_qmnist, key_ins_digit))       
    count_bags = 0; i_range_min=0; i_range_max=int(1e20)
    max_num_inst_of_type = num_key_ins_per_bag
    max_num_bags_of_type = np.ceil(num_bags/2)-1
    _, count_bags_pos = auxiliary_function(count_bags, i_range_min, i_range_max, max_num_bags_of_type, 
                        os.path.join(dir_qmnist, key_ins_digit), max_num_inst_of_type, key_instance_list, save_f)

    ''' Negative bags '''
    save_f = create_save_dir(save_subfolder, 'negative')
    i_range_max = int(1e20)
    count_bags = count_bags_pos
    max_num_bags_of_type = count_bags_pos+np.ceil(num_bags/2)-1
    
    _,_ = auxiliary_function(count_bags, last_used_index_in_list, i_range_max, max_num_bags_of_type, 
                                                 dir_qmnist, num_instances_in_bag, img_names_list, save_f)
    

In [ ]:
# Choose settings
number_of_folds = 1 # Select how many veraions of the dataset with the same parameters to create (here referred to as fold)
num_instances_in_bag = 1000 # Select number of instances in one bag
percent_key_instances = 10 # Select percent of key instances in positive bags
num_bags_train = 6 # Overall number of bags for training (num_bags_train/2 positive num_bags_train/2 negative)
num_bags_valid = 4 # Overall number of bags for validation
num_bags_test = 10 # Overall number of bags for test
for fold in range(1,number_of_folds+1):
    fold_dir = create_save_dir('./', 'fold'+str(fold)) 
    # Train
    dir_qmnist = './QMNIST_train/'
    save_folder = create_save_dir(fold_dir,'QMNIST'+'_'+str(num_bags_train).zfill(4)+'_'+
                                  str(num_instances_in_bag).zfill(4)+'_'+str(percent_key_instances).zfill(4))
    if not os.listdir(save_folder):    
        save_subfolder = create_save_dir(save_folder, 'train')
        compose_dataset(dir_qmnist, save_subfolder, num_bags_train, num_instances_in_bag, percent_key_instances)

        # Validation
        dir_qmnist = './QMNIST_valid/'
        save_subfolder = create_save_dir(save_folder, 'valid')
        compose_dataset(dir_qmnist, save_subfolder, num_bags_valid, num_instances_in_bag, percent_key_instances)

        # Test set 
        num_bags = num_bags_train
        dir_qmnist = './QMNIST_test/'
        save_subfolder = create_save_dir(save_folder, 'test')
        compose_dataset(dir_qmnist, save_subfolder, num_bags_test, num_instances_in_bag, percent_key_instances)